In [2]:
import numpy as np
import pandas as pd
from pyecharts.charts import Bar
from pyecharts import options as opts
from src.describe import summary_cluster, one_hot_encode_clusters, random_forest_train, render_importance_rank

設定cluster csv路徑並讀取資料

In [3]:
cluster_path = './cluster-result/1714460704_som_sklearn/cluster_20.csv'
cluster = pd.read_csv(cluster_path)

根據cluster進行劃分

In [4]:

chart = summary_cluster(cluster)
chart.render_notebook()

# Unsupervised to Supervised

將各分群的資料獨立並進行binary encoding

In [5]:
one_hot_encoded_clusters = one_hot_encode_clusters(cluster)

每個分群進行隨機森林建模 (Binary classification)

In [22]:
kappas = []
importance_bars = []
importance_data = {}
for cluster_index in range(len(one_hot_encoded_clusters)):
	feature_importance_df, kappa = random_forest_train(
		one_hot_encoded_clusters,
		cluster_index
	)
	importance_bar = render_importance_rank(
		feature_importance_df,
		cluster_index
	)
	kappas.append(kappa)
	importance_bars.append(importance_bar)
	importance_data[cluster_index] = {
		"feature": list(feature_importance_df['Feature']),
		"importance": list(feature_importance_df['Importance'])
	}
	


Cluster 0 | Accuracy: 0.9983866666666666 | Kappa: 0.9767765722613728
Cluster 1 | Accuracy: 0.9928 | Kappa: 0.9545203252641655
Cluster 2 | Accuracy: 0.9918133333333333 | Kappa: 0.9567336590288187
Cluster 3 | Accuracy: 0.9954133333333334 | Kappa: 0.9714522853796591
Cluster 4 | Accuracy: 0.9958 | Kappa: 0.9195473472562202
Cluster 5 | Accuracy: 0.99144 | Kappa: 0.9145867026868704
Cluster 6 | Accuracy: 0.9909866666666667 | Kappa: 0.9077614272566663
Cluster 7 | Accuracy: 0.9934933333333333 | Kappa: 0.9073162360151026
Cluster 8 | Accuracy: 0.99108 | Kappa: 0.93422304727518
Cluster 9 | Accuracy: 0.9900533333333333 | Kappa: 0.8622500953293353
Cluster 10 | Accuracy: 0.99244 | Kappa: 0.8885244812507378
Cluster 11 | Accuracy: 0.99212 | Kappa: 0.9170030435475452
Cluster 12 | Accuracy: 0.9920266666666666 | Kappa: 0.9611577362713456
Cluster 13 | Accuracy: 0.9928133333333333 | Kappa: 0.9646669948389777
Cluster 14 | Accuracy: 0.9920666666666667 | Kappa: 0.8994234784750017
Cluster 15 | Accuracy: 0.99297

畫出每個分群隨機森林的kappa

In [23]:
xy_pairs = list(zip(
	[cluster_index for cluster_index in range(len(one_hot_encoded_clusters))], 
	kappas
))
sorted_pairs = sorted(xy_pairs, key=lambda pair: pair[1])
xaxis_sorted, yaxis_sorted = zip(*sorted_pairs)
chart = Bar()
chart.add_xaxis(xaxis_sorted)
chart.add_yaxis("Kappa", yaxis_sorted)
chart.set_global_opts(
	title_opts=opts.TitleOpts(title="Cluster Random Forest Kappa")
)
chart.render_notebook()


劃出每個分群的feature importance (Bar and Heatmap)

In [1]:
from pyecharts.charts import Page, HeatMap

x = [cluster for cluster in importance_data.keys()]
y = importance_data[0]['feature']
value = []
max = 0
for cluster in importance_data.keys():
	for j, imp in enumerate(importance_data[cluster]['importance']):
		cellValue = imp * 100
		cell = [cluster, j, cellValue]
		value.append(cell)
		if cellValue > max:
			max = cellValue

c = (
	HeatMap()
	.add_xaxis(x)
	.add_yaxis("Feature Importance", y, value)
	.set_global_opts(
		title_opts=opts.TitleOpts(title="HeatMap"),
		visualmap_opts=opts.VisualMapOpts(
			max_=max,
			range_color=["#2b83ba","#abdda4", "#ffffbf", "#fdae61", "#d7191c" ]
		),
	)
)
c.render_notebook()


NameError: name 'importance_data' is not defined

In [31]:

page = Page(layout=Page.SimplePageLayout)

page.add(
	importance_bars[0],
	importance_bars[1],
	importance_bars[2],
	importance_bars[3],
	importance_bars[4],
	importance_bars[5],
	importance_bars[6],
	importance_bars[7],
	importance_bars[8],
	importance_bars[9],
	importance_bars[10],
	importance_bars[11],
	importance_bars[12],
	importance_bars[13],
	importance_bars[14],
	importance_bars[15]
)
page.render_notebook()